In [61]:
import pandas as pd

In [187]:
import pandas as pd
 
from sklearn.ensemble import *

class ordinalClassifier():
    def __init__(self):
        self.score_column = 'TARGET'
        self.algorithm = RandomForestClassifier()
        
        
       
   

    ## private method to be called inside fit
    def __train_models_for_score(self,x,y):
        dicio = {}
        
        lista = list(y.unique())
        lista.sort(reverse = False)
        print(lista)
        self.scores = lista
        
        for i in self.scores:
            print(i)
            new_y = y > i
            print(sum(new_y))
            modelo = self.algorithm.fit(x,new_y)
            dicio.update({i:modelo})
     
        return dicio


    def fit(self,x,y):
        self.min_score = int(y.min())
        self.max_score = int(y.max())
        print(self.min_score)
        print(self.max_score)
        model = self.__train_models_for_score(x,y)

        return model


    ## private methods to be called inside predict
    def __predict_greater_than_number_probabilities(self,x):
        preds = pd.DataFrame()

        for i,model in self.model_dict.items():
            x = x[model.feature_names_in_]
            a = model.predict_proba(x)
            preds[f'Pr(>{i})'] = model.predict_proba(x).T[0]

        return preds

    def __converting_probabilities(self,preds):
        new_preds = pd.DataFrame()

        lista = list(self.model_dict.keys())
        print(lista)
        min_score = lista[0]
        max_score = lista[-1] + 1
        lista = lista[1:]
        #print(lista)

        for i in range(len(lista)):
             
            new_preds[i] =  preds[f'Pr(>{(lista[i-1])})'] - preds[f'Pr(>{lista[i]})'] 

        new_preds[min_score] = 1 - preds[f'Pr(>{(min_score)})']
        new_preds[max_score] = preds[f'Pr(>{(max_score-1)})']

        return new_preds

    def __extracting_point_predictions(self,new_preds):
        y_preds = new_preds.to_numpy().argmax(axis = 1) 
        
        

        return y_preds

    def predict(self,x,model_dicio):            
        self.model_dict = model_dicio

        preds = self.__predict_greater_than_number_probabilities(x)
        print('TAOKEY')
        new_preds = self.__converting_probabilities(preds)
        
        print('TAOKEY')
        y_hats = self.__extracting_point_predictions(new_preds)
        
        x['pred'] = y_hats
        return preds,new_preds,x

In [188]:
TARGETS_1 = ['Domínio da modalidade escrita formal_nota',                                                                                                  
 'Selecionar, relacionar, organizar e interpretar informações em defesa de um ponto de vista_nota',                                                 
 'Compreender a proposta e aplicar conceitos das várias áreas de conhecimento para desenvolver o texto dissertativo-argumentativo em prosa_nota',   
'Conhecimento dos mecanismos linguísticos necessários para a construção da argumentação_nota',                                                     
'Proposta de intervenção com respeito aos direitos humanos_nota']           
ID_VARS = ['id','index','tema','conjunto']
EXCLUDE_COLS = TARGETS_1 + ID_VARS

In [189]:
df_train = catalog.load('vectorized_train_list')['LSI_32']()
df_test = catalog.load('vectorized_test_list')['LSI_32']()

[02/06/24 00:25:01] INFO     Loading data from 'vectorized_train_list' (PartitionedDataset)...  ]8;id=106265;file://C:\Users\ramon.boucas\.conda\envs\my_env\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=669374;file://C:\Users\ramon.boucas\.conda\envs\my_env\lib\site-packages\kedro\io\data_catalog.py#502\502]8;;\

                    INFO     Loading data from 'vectorized_test_list' (PartitionedDataset)...   ]8;id=993849;file://C:\Users\ramon.boucas\.conda\envs\my_env\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=69;file://C:\Users\ramon.boucas\.conda\envs\my_env\lib\site-packages\kedro\io\data_catalog.py#502\502]8;;\

In [190]:
df_train['TARGET'] = df_train[TARGETS_1].sum(axis=1)
df_train['TARGET'] = df_train['TARGET'].astype(int)
df_train = df_train.drop(columns = TARGETS_1)
df_train = df_train.drop(columns = ['texto'])
id_train = df_train[ID_VARS]
X_train = df_train.drop(columns = EXCLUDE_COLS + ['TARGET'],errors = 'ignore')
y_train = df_train['TARGET'].astype(float)


In [191]:
df_test['TARGET'] = df_test[TARGETS_1].sum(axis=1)
df_test['TARGET'] = df_test['TARGET'].astype(int)
df_test = df_test.drop(columns = TARGETS_1)
df_test = df_test.drop(columns = ['texto'])
id_test = df_test[ID_VARS]
X_test = df_test.drop(columns = EXCLUDE_COLS + ['TARGET'],errors = 'ignore')
y_test = df_test['TARGET'].astype(float)


In [192]:
oc = ordinalClassifier()
vasco = oc.fit(X_train,y_train)

200
960
[200.0, 360.0, 400.0, 440.0, 480.0, 520.0, 560.0, 600.0, 640.0, 680.0, 720.0, 760.0, 800.0, 840.0, 880.0, 920.0, 960.0]
200.0
6053
360.0
5884
400.0
5761
440.0
5626
480.0
5438
520.0
5160
560.0
4816
600.0
4471
640.0
3931
680.0
3627
720.0
3169
760.0
2707
800.0
2032
840.0
1486
880.0
895
920.0
212
960.0
0


In [185]:
a= oc.predict(X_test,vasco)

TAOKEY
[200.0, 360.0, 400.0, 440.0, 480.0, 520.0, 560.0, 600.0, 640.0, 680.0, 720.0, 760.0, 800.0, 840.0, 880.0, 920.0, 960.0]
TAOKEY


In [186]:
a[0]

,Pr(>200.0),Pr(>360.0),Pr(>400.0),Pr(>440.0),Pr(>480.0),Pr(>520.0),Pr(>560.0),Pr(>600.0),Pr(>640.0),Pr(>680.0),Pr(>720.0),Pr(>760.0),Pr(>800.0),Pr(>840.0),Pr(>880.0),Pr(>920.0),Pr(>960.0)
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1521,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1522,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1523,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1524,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [162]:
id_train

,id,index,tema,conjunto
0,11659,27,83,1
1,15612,57,39,1
2,14071,35,57,1
3,17269,23,26,1
4,16059,109,36,1
...,...,...,...,...
6097,14327,46,52,1
6098,17203,101,26,1
6099,14979,41,46,1
6100,18522,94,14,1


In [163]:
oc.scores

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 oc.scores                                                                                    │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'ordinalClassifier' object has no attribute 'scores'

In [36]:
vasco


{
    200.0: RandomForestClassifier(),
    360.0: RandomForestClassifier(),
    400.0: RandomForestClassifier(),
    440.0: RandomForestClassifier(),
    480.0: RandomForestClassifier(),
    520.0: RandomForestClassifier(),
    560.0: RandomForestClassifier(),
    600.0: RandomForestClassifier(),
    640.0: RandomForestClassifier(),
    680.0: RandomForestClassifier(),
    720.0: RandomForestClassifier(),
    760.0: RandomForestClassifier(),
    800.0: RandomForestClassifier(),
    840.0: RandomForestClassifier(),
    880.0: RandomForestClassifier(),
    920.0: RandomForestClassifier(),
    960.0: RandomForestClassifier()
}